## 271 - Getting Multispectral GOES ABI Imagery

[Youtube](https://www.youtube.com/watch?v=vs2xXsV0ZpA)

In [ ]:
from datetime import datetime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import metpy
from metpy.plots import add_timestamp
import numpy as np
from siphon.catalog import TDSCatalog

In [ ]:
def open_dataset(date, channel, idx, region):
    """
    Open and return dataset from GOES for given parameters.
    """
    cat = TDSCatalog(f'https://thredds.ucar.edu/thredds/catalog/satellite/goes/east/products/CloudAndMoistureImagery/{region}/Channel{channel:02d}/{date:%Y%m%d}/catalog.xml')
    ds = cat.datasets[idx]
    ds = ds.remote_access(use_xarray=True)
    return ds

In [ ]:
date = datetime(2023, 2, 2)
region = 'CONUS'
idx = 220

In [ ]:
red_ds = open_dataset(date, 2, idx, region)
veggie_ds = open_dataset(date, 3, idx, region)
blue_ds = open_dataset(date, 1, idx, region)

In [ ]:
red_ds

In [ ]:
data = {'red': np.clip(red_ds['Sectorized_CMI'][::2, ::2], 0, 1),
        'veggie': np.clip(veggie_ds['Sectorized_CMI'], 0, 1),
        'blue': np.clip(blue_ds['Sectorized_CMI'], 0, 1)}

In [ ]:
data['color_image'] = np.dstack((data['red'], data['veggie'], data['blue']))

In [ ]:
dat = blue_ds.metpy.parse_cf('Sectorized_CMI')
proj = dat.metpy.cartopy_crs
x = dat['x']
y = dat['y']

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, projection=proj)

ax.add_feature(cfeature.COASTLINE, linewidth=2)
ax.add_feature(cfeature.STATES, linestyle=':', edgecolor='black')
ax.add_feature(cfeature.BORDERS, linewidth=2, edgecolor='black')

im = ax.imshow(data['color_image'], extent=(x.min(), x.max(), y.min(), y.max()), origin='upper')

timestamp = datetime.strptime(blue_ds.start_date_time, '%Y%j%H%M%S')

add_timestamp(ax, time=timestamp, high_contrast=True,
              pretext=f'GOES 16 ',
              time_format='%d %B %Y %H%MZ', y=0.01,
              fontsize=18)